# Aqua 0.7 Operator Refactor
_5-Dec-19, donny@, ..._

## Purpose
To improve the transparency and ease of understanding of Qiskit’s operator logic and algorithmic usage. Specifically, to reconcile with the Terra operator hierarchy and make the ExpectationValue and Evolution algorithms more visible, explicit, and extensible.

## Background: Motivation and Opportunities
The representation of matrices sparsely as linear combinations of Pauli operators is critical in many quantum algorithms. As such, the Operator classes are the workhorses of Aqua today (0.6.2), containing both the expectation value and evolution logic used by most of its algorithms.

However, there are several opportunities for improvement:
* **Basic Construction & Rapid Protoyping:** Aqua's Operators were initially built as procedural infrastructure rather than first-class programming primitives. Improvements to syntax and interfaces can enable the succinctness and power typical of mathematical Operator language
* **Separation of Operator Math and Operator Algorithms**
    * Ease of understanding: The "Operator algorithm" logic - the ExpectationValue, Evolution, grouping, and symmetry analysis - is mostly spread across the 3000-line operator hierarchy, and is very branchy for different modes of execution
    * Ease of extension: Modification to the expectation value, evolution, grouping, and symmetry logic is a core use case (e.g. the [CVaR expectation](https://arxiv.org/abs/1907.04769), [linear combination evolution](https://arxiv.org/abs/1202.5822), or the many recent papers on [Pauli grouping](https://www.nature.com/articles/nature23879)), but not explicitly supported today
* **Smooth Borders with Broader Qiskit**
    * Terra's `quantum_info` module also supports operator math, but is fully matrix-based
    * WeightedPauli's ExpectationValue logic may be better housed under Ignis's tomography
    * **Remote Operator Algorithms:** Aer's fast ExpectationValue is not transparently or cleanly interchangeable with Aqua's local ExpectationValue today. The concept of an Algorithm not provided by Aqua is not yet defined to support this type of interchangeability cleanly

### Present State of Operators in Qiskit

Both Aqua and Terra include suites of modules to support Operator math, but do so very differently.

* Aqua
    * Operators are focused primarily on the procedural requirements of algorithmic execution
        * Modules are very large and include hundreds of lines of procedural algorithm code
        * Interfaces were not initial built for end-user usage as a programming primitive, and are therefore wordy and difficult for users to understand
        * Syntax is not built for rapid prototyping and lacks syntactic power of mathematical Operator language
    * Primarily focused on Pauli-basis Operators
        * WeightedPauli - $2^n\times 2^n$ Operators sparsely represented as complex combination of Paulis
        * MatrixOperator in the standard basis with $2^n\times 2^n$ elements was initially built for performance improvements which are no longer relevant
    * Only dependency on Terra is through Pauli module, but this is largely symbolic (not an inexorable component)
* Terra
    * Operator math is mostly built around QCVV and open Quantum systems modelling use cases
        * Support for Channel, Choi, Superoperator, Kraus, etc.
    * Operators are largely matrix-based and therefore do not support the Pauli-basis operations necessary to non-exponentially execute quantum algorithms
    * Used by: 
        * Aqua, 29 dependencies - Only Pauli module
        * Aer, 10 dependencies
        * Ignis, 2 dependencies

### Aqua Present Usage (0.6.2)

Within Aqua, the primary uses of Operators are:
* Qubit Observable (Hamiltonian, Cost Function, etc.) Construction
    * Used as sparse representations of large observables when constructing problems in Chemistry, Physics, Optimization, and Finance today
    * Also often a translation step between domain-specific problems and Quantum hardware-addressable equivalents
* ExpectationValues
    * Primarily used in VQE (and derivatives) as a device-executable cost function of the ansatz state
    * Also present in the "Evolution of Hamiltonian" algorithm, which is simply state evolution by one operator followed by an expectation value by another operator
    * Expectation values can only be taken of Operators in the Puali basis on Quantum hardware
* State Evolution
    * Used in QPE (and derivatives AE, HHL, iQPE, etc.) as a Quantum circuit-representable matrix exponentiation
    * Used in UCCSD and QAOA ansatze and EOH algorithm as representation of system dynamics to simulate time evolution of a system on quantum hardware
    * Evolution can only be taken by Operators in the Puali basis on Quantum hardware

### Aqua Present (0.6.2) Operator Object Model and Hierarchy

Aqua's Operators are organized as follows:
* `qiskit.aqua.operators`
    * base_operator.py: `BaseOperator(ABC)`
    * matrix_operator.py: `MatrixOperator(BaseOperator)`
    * weighted_pauli_operator.py: `WeightedPauliOperator(BaseOperator)`, __and__ `Z2Symmetries`
    * tpb_grouped_weighted_pauli_operator.py: `TPBGroupedWeightedPauliOperator(WeightedPauliOperator)`, essentially a wrapper around `WeightedPauliOperator` for backward compatibility.
    * pauli_graph: `PauliGraph`
    * op_converter.py: `to_weighted_pauli_operator(operator)`, `to_matrix_operator(operator)`, `to_tpb_grouped_weighted_pauli_operator(operator, grouping_func, **kwargs)`
    * common.py: Utility functions, inc. `evolution_instruction`, `pauli_measurement(circuit, pauli, qr, cr, barrier=False)`, `measure_pauli_z(data, pauli)`, `covariance(data, pauli_1, pauli_2, avg_1, avg_2)`, etc.
* `qiskit.chemistry` __OUT OF SCOPE OF THIS DESIGN DOC__
    * fermionic_operator.py: `FermionicOperator`, contains `jordan_wigner`, `parity`, `bravyi_kitaev` Fermion-to-qubit operator mappings.
    * bksf.py: Another mapping
    * `.core`
        * chemistry_operator.py: `ChemistryOperator(ABC)`
        * hamiltonian.py: `Hamiltonian(ChemistryOperator)`

_Although many modules in optimization and finance construct Ising Hamiltonians as cost function observables, there is no Ising module._

### Terra Present (0.11.0) Operator Object Model and Hierarchy

Terra's Operators are organized as follows:
* `qiskit.quantum_info`
    * `.operators`
        * base_operator.py, pauli.py, operator.py (matrix operator), measures.py (`process_fidelity`), predicates.py (`is_unitary_matrix`, `is_hermitian_matrix`, `matrix_equal`, etc.), quaternion.py
        * `.channel`
            * quantum_channel.py (base), chi.py, choi.py, kraus.py, ptm.py, stinespring.py, superop.py, transformations.py
    * `.states`
        * quantum_state.py (base), densitymatrix.py, statevector.py, measures.py (`state_fidelity`), states.py (`basis_state`, `projector`, `purity`)
    * `.analysis`
        * average.py - ExpectationValue of diagonal operator
        * make_observable.py - Convert an observable in matrix form to dictionary form

## Requirements and Design

### Construction and Manipulation

In [3]:
# from qiskit.quantum_info.operators import WeightedPauliOperator
from qiskit.aqua.operators import WeightedPauliOperator, MatrixOperator, op_converter
from qiskit.quantum_info.operators import Pauli

#### Present State

Today, Qiskit supports several methods of WeightedPauli operator construction:

In [4]:
pauli_op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
    [.1j, Pauli.from_label('ZZ')],
])

In [5]:
pauli_op = WeightedPauliOperator.from_list(
    paulis=[Pauli.from_label('IX'),
            Pauli.from_label('ZY'),
            Pauli.from_label('ZZ')],
    weights=[.5, .2, .1j])

In [6]:
mat = [[0. +0.1j, 0.5-0.2j, 0. +0.j , 0. +0.j ],
       [0.5+0.2j, 0. -0.1j, 0. +0.j , 0. +0.j ],
       [0. +0.j , 0. +0.j , 0. -0.1j, 0.5+0.2j],
       [0. +0.j , 0. +0.j , 0.5-0.2j, 0. +0.1j]]
mat_op = MatrixOperator(mat)
pauli_op_from_mat = op_converter.to_weighted_pauli_operator(mat_op)
pauli_op == pauli_op_from_mat

True

Classical matrices can be exported if the user already knows the Operator hierarchy somewhat well:

In [7]:
op_converter.to_matrix_operator(pauli_op).matrix.toarray()

array([[0. +0.1j, 0.5-0.2j, 0. +0.j , 0. +0.j ],
       [0.5+0.2j, 0. -0.1j, 0. +0.j , 0. +0.j ],
       [0. +0.j , 0. +0.j , 0. -0.1j, 0.5+0.2j],
       [0. +0.j , 0. +0.j , 0.5-0.2j, 0. +0.1j]])

Addition, scalar multiplication, composition, and deep equality are available for _Operators of the same representation_. However, composition uses the `*` operator, while Terra's operators and Python use `@`. Printing defaults to 

In [13]:
3*pauli_op + .2j*pauli_op == (3+.2j)*pauli_op

True

In [33]:
print((pauli_op * pauli_op).print_details())

II	(0.28+0j)
ZZ	0j
ZY	0j
IX	0j



In [9]:
pauli_op == mat_op

AttributeError: 'MatrixOperator' object has no attribute 'simplify'

#### [P1] Mathematical Construction

The above code lacks the mathematical structure or power of typical operator expressions, and lacks a self-explanatory relationship to the underlying Paulis used. As such, Qiskit should support the following (note that `^` is already implemented as the overload for kron product in Terra):

In [2]:
from qiskit.aqua.operators.pauli import X, Y, Z, I

In [9]:
op_new = .5*(I^X) + .2*(Z^Y) + .1j*(Z^Z)
op_new == pauli_op

True

Note that to support the above, Pauli and WeightedPauli must support the following:
* Tensor, sum, and scalar multiplication without converting to matrices
* Operations which return Operators outside of the Pauli group
* Deep equality evaluation between WeightedPauli operators
* A default set of simple operator instances

#### [P1] Operator Math Overloads, Richness and Consistency

The following overload operations are desirable:
* Operator composition using `@` overload, deprecate the `*` overload for composition
* Power (`**3`), kronpower (`^3`)

In [34]:
(pauli_op^2)**2 == (pauli_op^pauli_op)@(pauli_op^pauli_op)

True

#### Other Usage Syntactic Sugar

In addition, the following operations are desirable:
* `to_matrix()` method to allow quick access to unscalable classical tools, e.g. numpy eigensolution
* Trace, performed recursively through Paulis

#### Future Work: Other Important Qubit Operator Bases, MixedOperator

Operators can also be constructed as complex combinations of Cliffords, Clifford+Ts, Projectors, Stabilizers, QuantumCircuits, and ZX Hamiltonians. Qiskit should support these primitives to allow more powerful Operator compositions. Qiskit should consider supportting a MixedOperator, holding a complex combination of various types of operators.

#### Grouping

### ExpectationValue Algorithms

#### Present State

Aqua's ExpectationValue is not contained within a single function or module, but rather split into several functions without a clear interface or flow for user usage. This is due to structural constraints in Aqua which are no longer present, where the algorithm requiring the expectation value held the backend object and could run circuits, but the operator could not. We encourage the reader to scan lines [361-395 of Aqua 6.1 VQE’s](https://github.com/Qiskit/qiskit-aqua/blob/stable/qiskit/aqua/algorithms/adaptive/vqe/vqe.py#L361) ExpectationValue calculation to try to understand where and how the expectation is computed. We’ve been asked by numerous Aqua users to explain how this code works, and most do not attempt to use it on their own.

The following is the shortest possible way to write an expectation value in Aqua. Note that it fundamentally requires the user to understand a certain execution flow, the correct functions to use to do this, and how those functions work with their execution mode. This takes a few hours to understand at least, often days. Further, there are no hints that tomography is being performed here, or matrix multiplication if the system chooses to do that instead.

In [4]:
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.variational_forms import RY
from qiskit.quantum_info import Pauli
from qiskit import BasicAer, execute, QuantumCircuit
from qiskit.circuit import Parameter
qasm_sim = BasicAer.get_backend('qasm_simulator')

In [2]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2j, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)
circuit.h([0,1])

evaluation_circuits = op.construct_evaluation_circuit(wave_function=circuit, statevector_mode=False)
result = execute(evaluation_circuits, qasm_sim).result()
expect, std = op.evaluate_with_result(result=result, statevector_mode=False)
expect

(0.5+0.005078125j)

### State Evolution Algorithms

#### Present State

Evolution is somewhat more succinct, but more difficult to navigate in code.

In [4]:
from qiskit.circuit import Parameter

In [5]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)

θ = Parameter('θ')
instr = op.evolve_instruction(evo_time=θ)
circuit.append(instr, [0,1])
print(circuit.draw(fold=4000))
print('Decomposed:')
circuit.decompose().draw(fold=4000)

        ┌─────────────────┐
q_0: |0>┤0                ├
        │  Evolution^1(θ) │
q_1: |0>┤1                ├
        └─────────────────┘
Decomposed:


# ⚰️⚰️⚰️⚰️⚰️⚰️ Graveyard ⚰️⚰️⚰️⚰️⚰️⚰️

#### Tasks Not Possible Today

* Explicitly using Aer's ExpectationValue, or explicitly disabling it
* Modifying the ExpectationValue, Evolution, grouping, or symmetry code without copying a significant amount of code

### WeightedPauliOperator Not Available in Terra

Terra contains operator utilities in the `qiskit.quantum_info.operators` directory, focused primarily on the mathematical operations between matrix-represented operators of various types (e.g. Choi, Pauli, etc.). Terra's operators do not contain much of the Pauli-basis logic in Aqua today, and are not interoptable with Aqua's operator algorithms. As such, these utilities are only accessible to Aqua users.

### Ignis and Aer Logic Living in Aqua

* Aqua's ExpectationValue relies on a form of tomography, which in principle should live within Ignis's tomography utilities.
* Aer's ExpectationValue is a super-superuser feature today
    * The interface (and existence) of the Aer-provided algorithm is not made obvious because there is no way to specify a non-Aqua-provided algorithm in Qiskit
    * In Aqua, there is no simple way to specify which ExpectationValue algorithm the user wants. 
    * Aer's ExpectationValue is woven throughout the core operator code in a way that is branchy, inexorable, and difficult for users to understand and control.

## Core Requirements / Interface

1. Broadly Available (Qiskit-wide) WeightedPauli Logic
    1. Basic Construction and Usage
    1. Interface, Variants, Organization
        1. Option A: WeightedPauli class in Terra
        1. Option B: WeightedUnitary class in Terra
    1. Integration Testing
1. ExpectationValue and Evolution algorithm hierarchies
    1. Interface, Variants, Organization
        1. Usage of Operator Abstractions within Aqua Algorithms
        1. Usage of Operator Abstractions without Aqua Algorithms
    1. ExpectationValue Reconciliation with Ignis tomography
        1. Option A: Leave ExpectationValue in Aqua
        1. Option B: Keep ExpectationValue algo in Aqua, move logic into Ignis
        1. Option C: Move ExpectationValue algo into Ignis
1. Aer ExpectationValue and Evolution
    1. Option I: Leave AerExpectation in Aqua
    1. Option II: Central Qiskit Algorithms base classes
    1. Option III: Merge Aqua and Terra
1. Smooth Borders with Applications Primitives - e.g. FermionicOperator
    1. Move Operator Translation Into Application Stacks, Consistently

### Broadly Available (Qiskit-wide) WeightedPauli Logic

A **WeightedPauliOperator** is an n-qubit operator represented sparesely as a complex combination of n-qubit Pauli operators (tensor products of n single-qubit Paulis). All hermitian operators can be represented in this basis.

Many quantum algorithms depend on this sparse representation of operators in the Pauli basis. Terra contains a suite of Matrix-represented operators within the quantum_info module, including rich logic for combining and manipulating various operator types, but does not include a WeightedPauli operator. 
* The Aqua operator does not need to replicate Terra's logic within MatrixOperator, and should instead rely on Terra for Matrix-based operator logic. 
* Aqua is the not the only place in Qiskit which requires WeightedPauli Operators. These are used across quantum information, and should be interoptable with the other existing Terra operators.

**Requirement:** The mathematical (non-algorithmic) Operator logic in Aqua should be moved into Terra, and Aqua should use these as its core operator objects.

### Breaking out the ExpectationValue and Evolution to be explicit modules which can be interchanged and extended.

Users want to easily find and understand ExpectationValue and Evolution logic, and understand the algorithmic variations available to them.

...

In [ ]:
from qiskit.aqua.algorithms import ExpectationValue, Evolution

avg = ExpectationValue(operator, backend).run(circuit)
instr = Evolution(operator, backend).instruction(param)

### Terse and transparent Operator usage within the Aqua algorithms, characteristic of end-user usage of the same algorithms.

...

In [ ]:
class VQE(QuantumAlgorithm):
    def __init__(self, operator, var_form, optimizer,
                 initial_point=None, backend=backend, callback=None, ...):
        ...
        self._expectation_value = ExpectationValue(self._operator, self._backend)
    
    def _energy_evaluation(self, params):
        circuits = self._var_form.construct_circuit(params)
        energy, stdev = self._expectation_value.run(circuits)
        return energy

### Explicit modules for Aer's ExpectationValue and Evolution which can be swapped for Aqua's, and a framework for remote backends to provide RemoteAlgorithm modules without depending on Aqua. 

...

In [ ]:
from qiskit.providers.aer import AerExpectationValue

my_vqe = VQE()

### Reconciling the implicit tomography in Aqua's expectation value with Ignis's tomography functionality to avoid code duplication and allow generic access to the functionality.

...

### Clear and smooth borders with the Operator-related objects in Aqua, such as the FermionicOperator and Hamiltonian in Chemistry, and the proposed LadderOperator and OptimizationProblem.

...

## Technical Design

* Changes to Terra Operator Hierarchy
* Aqua Algorithm Hierarchy
    * Groups and Symmetries Options
* Changes to Ignis
* Qiskit Remote Algorithms
* Aer ExpectationValue and Evolution
* IBMQ or other backend RemoteAlgorithms

### Changes to Terra Operator Hierarchy

There are two options for the migration of the Operator mathematical logic into Terra's quantum_info:

#### Option A: Move WeightedPauliOperator into quantum_info and refactor

...

#### Option B: Add Linear Combination functionality to Operator to allow for WeightedPauli, WeightedClifford, and Combinations

* e.g. Pauli a + Pauli b = LinComb(a, b)
* Circuit Operator?

...

...

## Timeline and Gameplan

...

## Future Extension Opportunities